In [1]:
import pandas as pd 

df = pd.read_excel("Raisin_Dataset.xlsx")
df.sample(5)

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
602,74966,465.360816,209.279646,0.893172,78159,0.712415,1157.089,Besni
625,96210,482.512019,262.674411,0.838833,102157,0.686587,1270.564,Besni
210,49063,294.748287,217.913109,0.673355,50732,0.658811,869.795,Kecimen
652,98464,488.215004,261.724260,0.844165,101977,0.679432,1259.069,Besni
125,54316,351.805261,198.417520,0.825776,55960,0.655895,930.627,Kecimen


In [2]:
X = df[["Area", "MajorAxisLength", "MinorAxisLength", "Eccentricity", "ConvexArea", "Extent", "Perimeter"]]
y = df["Class"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Scaling

In [3]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler
scaler = StandardScaler()
scaler.fit(X)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
X_train.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
687,129292,614.547426,270.085490,0.898249,133475,0.689690,1531.744
705,79975,439.312508,240.494190,0.836849,83384,0.691162,1162.608
296,56327,297.765725,241.934426,0.582962,58178,0.727739,915.496
172,58741,345.485448,222.381845,0.765295,60701,0.714819,948.233
863,67468,424.563654,207.823052,0.872004,70674,0.611694,1105.042


In [5]:
X_train_scaled

array([[ 1.06432546,  1.58331003,  0.3121899 , ...,  1.03785038,
        -0.18372849,  1.33706534],
       [-0.20084761,  0.07228172, -0.28009664, ..., -0.19147815,
        -0.15618566, -0.01205588],
       [-0.80751089, -1.1482581 , -0.25126952, ..., -0.81008139,
         0.52829137, -0.9152028 ],
       ...,
       [ 2.02280624,  1.52682193,  1.90489827, ...,  1.91117405,
         1.01069031,  1.49210229],
       [-1.52866749, -1.59978989, -2.08647336, ..., -1.49386793,
        -1.44493478, -1.76491643],
       [-0.76951747, -1.04300791, -0.26263666, ..., -0.79103687,
         0.20282377, -0.92850265]])

#### WITHOUT SCALING

In [6]:
from sklearn.svm import SVC 

model = SVC(kernel='rbf')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import classification_report

report = classification_report(y_test,y_pred)
print(report)

model.n_iter_

              precision    recall  f1-score   support

       Besni       0.86      0.83      0.84       129
     Kecimen       0.85      0.87      0.86       141

    accuracy                           0.85       270
   macro avg       0.85      0.85      0.85       270
weighted avg       0.85      0.85      0.85       270



array([203], dtype=int32)

### with Scaling

In [7]:
from sklearn.svm import SVC 

model = SVC(kernel='rbf')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

from sklearn.metrics import classification_report

report = classification_report(y_test,y_pred)
print(report)

model.n_iter_

              precision    recall  f1-score   support

       Besni       0.87      0.86      0.86       129
     Kecimen       0.87      0.88      0.88       141

    accuracy                           0.87       270
   macro avg       0.87      0.87      0.87       270
weighted avg       0.87      0.87      0.87       270



array([343], dtype=int32)

## ENSEMBLE LEARNING

In [23]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

log_model = LogisticRegression()
svc_model = SVC(kernel='rbf', probability=True)
dt_model = DecisionTreeClassifier()

vc = VotingClassifier(estimators=[
    ('lr', log_model),
    ('svc', svc_model),
    ('dt', dt_model)
], voting='soft')

vc.fit(X_train, y_train)

y_pred = vc.predict(X_test)
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

       Besni       0.85      0.86      0.86       129
     Kecimen       0.87      0.87      0.87       141

    accuracy                           0.86       270
   macro avg       0.86      0.86      0.86       270
weighted avg       0.86      0.86      0.86       270



C:\Users\saisa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
